# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 29 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2212.14291


extracting tarball to tmp_2212.14291...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.14498


extracting tarball to tmp_2212.14498...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.14740


extracting tarball to tmp_2212.14740... done.
Retrieving document from  https://arxiv.org/e-print/2212.14824


extracting tarball to tmp_2212.14824...

 done.
Retrieving document from  https://arxiv.org/e-print/2212.14835


/tmp/ipykernel_2192/4030337529.py:34: LatexWarning: 2212.14835 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14291-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14291) | **Constraints on Dark Energy Models with Gamma-Ray Bursts Calibrated from  the Observational $H(z)$ Data**  |
|| <mark>Zihao Li</mark>, <mark>Bin Zhang</mark>, <mark>Nan Liang</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| *7 pages, 8 figures, 3 Tables. arXiv admin note: text overlap with arXiv:2211.02473*|
|**Abstract**| We use a cosmology-independent method to calibrate gamma-ray burst (GRB) from the observational Hubble data (OHD) which obtained with the cosmic chronometers method. By using Gaussian Process to reconstruct OHD, we calibrate the Amati relation ($E_{\rm p}$--$E_{\rm iso}$) to construct a GRB Hubble diagram with the A118 data set, and constrain Dark Energy models in a flat space with the Markov Chain Monte Carlo numerical method. With the cosmology-independent GRBs at $1.4<z\leq8.2$ in the A118 data set and the Pantheon sample of type Ia supernovae (SNe Ia) at $0.01<z\leq2.3$, we obtained $\Omega_{\rm m}$ = $0.398^{+0.028}_{-0.021}$, $h$ = $0.702^{+0.0035}_{-0.0035}$, $w$ = $-1.33^{+0.14}_{-0.12}$, $w_a$ = $-0.97^{+0.57}_{-0.57}$ for the flat CPL model at the 1$\sigma$ confidence level. We find that the $\Lambda$CDM model is favoured respect to the $w$CDM model and the CPL model with the selection criteria. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14498-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14498) | **The deformation of an erupting magnetic flux rope in a confined solar  flare**  |
|| Ruisheng Zheng, et al. -- incl., <mark>Yihan Liu</mark>, <mark>Liang Zhang</mark>, <mark>Yang Liu</mark>, <mark>Qing Lin</mark>, <mark>Libo Zhong</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| **|
|**Abstract**| Magnetic flux ropes (MFRs), sets of coherently twisted magnetic field lines, are believed as core structures of various solar eruptions. Their evolution plays an important role to understand the physical mechanisms of solar eruptions, and can shed light on adverse space weather near the Earth. However, the erupting MFRs are occasionally prevented by strong overlying magnetic fields, and the MFR evolution during the descending phase in the confined cases is lack of attention. Here, we present the deformation of an erupting MFR accompanied by a confined double-peaked solar flare. The first peak corresponded to the MFR eruption in a standard flare model, and the second peak was closely associated with the flashings of an underlying sheared arcade (SA), the reversal slipping motion of the L-shaped flare ribbon, the falling of the MFR, and the shifting of top of filament threads. All results suggest that the confined MFR eruption involved in two-step magnetic reconnection presenting two distinct episodes of energy release in the flare impulsive phase, and the latter magnetic reconnection between the confined MFR and the underlying SA caused the deformation of MFR. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14539-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14539) | **Low-Surface-Brightness Galaxies are missing in the observed Stellar Mass  Function**  |
|| Juhan Kim, et al. -- incl., <mark>Jaehyun Lee</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| *22 pages, 24 figures, submitted to ApJ*|
|**Abstract**| We investigate the impact of the surface brightness (SB) limit on the galaxy stellar mass functions (GSMFs) using mock surveys generated from the Horizon Run 5 (HR5) simulation. We compare the stellar-to-halo-mass relation, GSMF, and size-stellar mass relation of the HR5 galaxies with empirical data and other cosmological simulations. The mean SB of simulated galaxies are computed using their effective radii, luminosities, and colors. To examine the cosmic SB dimming effect, we compute $k$-corrections from the spectral energy distributions of individual simulated galaxy at each redshift, apply the $k$-corrections to the galaxies, and conduct mock surveys based on the various SB limits. We find that the GSMFs are significantly affected by the SB limits at a low-mass end. This approach can ease the discrepancy between the GSMFs obtained from simulations and observations at $0.625\le z\le 2$. We also find that a redshift survey with a SB selection limit of $\left<\mu_r\right>^e =$ 28 mag arcsec${}^{-2}$ will miss 20% of galaxies with $M_\star^g=10^{9}~{\rm M_\odot}$ at $z=0.625$. The missing fraction of low-surface-brightness galaxies increases to 50%, 70%, and 98% at $z=0.9$, 1.1, and 1.9, respectively, at the SB limit. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14602-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14602) | **Influence of magnetic reconnection on the eruptive catastrophes of  coronal magnetic flux ropes**  |
|| <mark>Quanhao Zhang</mark>, et al. -- incl., <mark>Rui Liu</mark>, <mark>Xiaolei Li</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| *This paper has been accepted by Frontiers in Astronomy and Space Sciences*|
|**Abstract**| Large-scale solar eruptive activities have a close relationship with coronal magnetic flux ropes. Previous numerical studies have found that the equilibrium of a coronal flux rope system could be disrupted if the axial magnetic flux of the rope exceeds a critical value, so that the catastrophe occurs, initiating the flux rope to erupt. Further studies discovered that the catastrophe does not necessarily exist: the flux rope system with certain photospheric flux distributions could be non-catastrophic. It is noteworthy that most previous numerical studies are under the ideal magnetohydrodynamic (MHD) condition, so that it is still elusive whether there is the catastrophe associated with the critical axial flux if magnetic reconnection is included in the flux rope system. In this paper, we carried out numerical simulations to investigate the evolutions of coronal magnetic rope systems under the ideal MHD and the resistive condition. Under the ideal MHD condition, our simulation results demonstrate that the flux rope systems with either too compact or too weak photospheric magnetic source regions are non-catastrophic versus varying axial flux of the rope, and thus no eruption could be initiated; if there is magnetic reconnection in the rope system, however, those flux rope systems could change to be capable of erupting via the catastrophe associated with increasing axial flux. Therefore, magnetic reconnection could significantly influence the catastrophic behaviors of flux rope system. It should be both the magnetic topology and the local physical parameters related to magnetic reconnection that determine whether the increasing axial flux is able to cause flux rope eruptions. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14712-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14712) | **Foreground Removal of CO Intensity Mapping Using Deep Learning**  |
|| Xingchen Zhou, et al. -- incl., <mark>Meng Zhang</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| *12 pages, 12 figures, 2 tables*|
|**Abstract**| Line intensity mapping (LIM) is a promising probe to study star formation, the large-scale structure of the Universe, and the epoch of reionization (EoR). Since carbon monoxide (CO) is the second most abundant molecule in the Universe except for molecular hydrogen ${\rm H}_2$, it is suitable as a tracer for LIM surveys. However, just like other LIM surveys, CO intensity mapping also suffers strong foreground contamination that needs to be eliminated for extracting valuable astrophysical and cosmological information. In this work, we take $^{12}$CO($\it J$=1-0) emission line as an example to investigate whether deep learning method can effectively recover the signal by removing the foregrounds. The CO(1-0) intensity maps are generated by N-body simulations considering CO luminosity and halo mass relation, and we discuss two cases with median and low CO signals by comparing different relations. We add foregrounds generated from real observations, including thermal dust, spinning dust, free-free, synchrotron emission and CMB anisotropy. The beam with sidelobe effect is also considered. Our deep learning model is built upon ResUNet, which combines image generation algorithm UNet with the state-of-the-art architecture of deep learning, ResNet. The principal component analysis (PCA) method is employed to preprocess data before feeding it to the ResUNet. We find that, in the case of low instrumental noise, our UNet can efficiently reconstruct the CO signal map with correct line power spectrum by removing the foregrounds and recovering PCA signal loss and beam effects. Our method also can be applied to other intensity mappings like neutral hydrogen 21cm surveys. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14740-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14740) | **The rotation rate of solar active and ephemeral regions -- II. Temporal  variations of the rotation rates**  |
|| Alexander S. Kutsenko, Valentina I. Abramenko, <mark>Daria V. Litvishko</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| *10 pages, 5 figures, accepted by MNRAS*|
|**Abstract**| Systematic studies of the rotation rate of sunspot groups using white-light images yield controversial results on the variations of the rotation rate: sunspot groups were found to either accelerate or decelerate systematically. This disagreement might be related to shortcomings of the method used to probe the rotation rate of sunspot groups. In contrast to previous works, in this study we use magnetic field maps to analyse the variations of the rotation rate of active regions. We found that an active region may exhibit either acceleration or deceleration during the emergence while the rotation rate remains almost unchanged during decay. Hence, we suppose that there is no systematic geometrical inclination to the radial direction of the apex of the subsurface magnetic flux loop forming an active region. A thorough comparison of the rotation rate of unipolar and bi/multipolar active regions revealed no significant changes in the rotation rate of decaying active regions. In contrast to previous works, we presume the rotation rate to keep constant (within the expected uncertainties) during the evolution of an active region after emergence. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14824-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14824) | **eDIG-CHANGES I: Extended Hα Emission from the Extraplanar Diffuse  Ionized Gas (eDIG) around CHANG-ES Galaxies**  |
|| <mark>Li-Yuan Lu</mark>, et al. -- incl., <mark>Jiang-Tao Li</mark>, <mark>George H. Heald</mark>, <mark>Hui Li</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| **|
|**Abstract**| The extraplanar diffuse ionized gas (eDIG) represents the cool/warm ionized gas reservoir around galaxies. We present a spatial analysis of H$\alpha$ images of 22 nearby edge-on spiral galaxies from the CHANG-ES sample (the eDIG-CHANGES project), taken with the APO 3.5m telescope, in order to study their eDIG. We conduct an exponential fit to the vertical intensity profiles of the sample galaxies, of which 16 can be decomposed into a thin disk plus an extended thick disk component. The median value of the scale height (h) of the extended component is $1.13\pm 0.14$ kpc. We find a tight sublinear correlation between h and the SFR. Moreover, the offset of individual galaxies from the best-fit SFR-h relation shows significant anti-correlation with SFR_SD. This indicates that galaxies with more intense star formation tend to have disproportionately extended eDIG. Combined with data from the literature, we find that the correlations between the eDIG properties and the galaxies' properties extend to broader ranges. We further compare the vertical extension of the eDIG to multi-wavelength measurements of other CGM phases. We find the eDIG to be slightly more extended than the neutral gas (HI 21-cm line), indicating the existence of some extended ionizing sources. Most galaxies have an X-ray scale height smaller than the h, suggesting that the majority of the X-ray emission detected in shallow observations are actually from the thick disk. The h is comparable to the L-band radio continuum scale height, both slightly larger than that at higher frequencies (C-band), where the cooling is stronger and the thermal contribution may be larger. The comparable H$\alpha$ and L-band scale height indicates that the thermal and non-thermal electrons have similar spatial distributions. This further indicates that the thermal gas, the cosmics rays, and the magnetic field may be close to energy equipartition. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2212.14835-b31b1b.svg)](https://arxiv.org/abs/arXiv:2212.14835) | **Mid-Infrared Spectroscopy of Components in Chondrites: Search for  Processed Materials in Young Solar Systems and Comets**  |
|| A. Morlok, et al. -- incl., <mark>C.M. Lisse</mark> |
|*Appeared on*| *2023-01-02*|
|*Comments*| **|
|**Abstract**| We obtained mid-infrared spectra of chondrules, matrix, CAIs and bulk material from primitive type 1-4 chondrites in order to compare them with the dust material in young, forming solar systems and around comets. Our aim is to investigate whether there are similarities between the first processed materials in our early Solar System and protoplanetary disks currently forming around other stars. Chondrule spectra can be divided into two groups. 1) Chondrules dominated by olivine features at 11.3 micron and 10.0 micron. 2) mesostasis rich chondrules that show main features at 10 micron. Bulk ordinary chondrites show similar features to both groups. Fine-grained matrix is divided into three groups. 1) phyllosilicate-rich with a main band at 10 micron, 2) olivine-rich with bands at 11.3 micron and 10 micron, 3) pyroxene rich. Impact shock processed matrix from Murchison (CM2) shows features from phyllosilicate-rich, amorphous and olivine rich material. Astronomical spectra are divided into four groups based on their spectral characteristics, amorphous (group 1), pyroxene rich (group 2), olivine rich (group 3) and complex (group 4). Group 2 is similar to enstatite-rich fine grained material like e.g. Kakangari (K3) matrix. Group 3 and 4 can be explained by a combination of varying concentrations of olivine and mesostasis rich chondrules and fine grained matrix, but also show very good agreement with shock processed material. Comparison of band ratios confirms the similarity with chondritic material e.g. for HD100546, while the inner disk of HD142527 show no sign of chondrule material. Comparison between spectra indicate a general similarity between primitive solar system materials and circumstellar dust and comets. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

148  publications in the last 7 days.
	 _build/html/2212.12039.md
	 _build/html/2212.11991.md
	 _build/html/2212.11425.md
	 _build/html/2212.11177.md
	 _build/html/2212.10804.md
	 _build/html/2212.10512.md
	 _build/html/2212.09766.md
	 _build/html/2212.09675.md
	 _build/html/2212.09661.md
	 _build/html/2212.09652.md
	 _build/html/2212.09443.md
	 _build/html/2212.09307.md
	 _build/html/2212.09168.md
	 _build/html/2212.09159.md
	 _build/html/2212.08702.md
	 _build/html/2212.07332.md
	 _build/html/2212.06434.md
	 _build/html/2212.04710.md
	 _build/html/2212.04026.md
	 _build/html/2212.03563.md
	 _build/html/2212.03514.md
	 _build/html/2212.02667.md
	 _build/html/2212.02627.md
	 _build/html/2212.02235.md
	 _build/html/2212.01707.md
	 _build/html/2212.01526.md
	 _build/html/2212.01397.md
	 _build/html/2212.01291.md
	 _build/html/2212.01281.md
	 _build/html/2212.00812.md
	 _build/html/2212.00811.md
	 _build/html/2212.00806.md
	 _build/html/2212.00047.md
	 _build/html/2212.00032.md
	 _build/h

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers